# 3. Model Building

##  Creating and Tuning ML Classifier

In [1]:
# import libraries
import numpy as np
import pandas as pd
import time
import re
import joblib 

import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet, stopwords
nltk.download('stopwords')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jamie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Read in data and tasting_note_list
df = pd.read_csv('./data/branded.csv', index_col = 'Unnamed: 0')
df_sms = pd.read_csv('./data/branded_singlemalts.csv', index_col = 'Unnamed: 0')

with open('./data/tasting_notes.txt', 'r') as f:
    wordlist = [item[:-1] for item in f.readlines()]

len(wordlist), df.shape, df_sms.shape

(1985, (27513, 7), (18868, 7))

In [3]:
# dropping brands that only have a frequency of lower_limit or fewer because it screws
# up cross-validation.
def drop_unpopular_brands(name, df, lower_limit=0):
    print("Dataframe {} had {} rows. After dropping rows with {} or fewer reviews, ".format(
        name, df.shape[0], lower_limit), end="")
    dfbrandcounts = df['brand'].value_counts()
    limited_list = dfbrandcounts[dfbrandcounts>lower_limit].index
    df = df[df['brand'].isin(limited_list)]
    print("it now has {} rows.".format(df.shape[0]))
    return df

lower_limit = 10
df = drop_unpopular_brands('df', df, 10)
df_sms = drop_unpopular_brands('df_sms', df_sms, 10)

Dataframe df had 27513 rows. After dropping rows with 10 or fewer reviews, it now has 26953 rows.
Dataframe df_sms had 18868 rows. After dropping rows with 10 or fewer reviews, it now has 18780 rows.


In [4]:
def tokenise_and_stem_text(text):
    '''
    INPUTS:
    text (string) - what you want to be lemmatised
    OUTPUTS:
    lemmas (list) - list of lemmatised next
    '''
    # Import stopword list and update with a few of my own
    stopword_list = stopwords.words("english")
    [stopword_list.append(i) for i in ['nose', 'palate', 'taste', 'finish']]
    
    # Normalise text - remove numbers too as we don't need them
    text = re.sub(r"[^a-zA-Z]", " ", text.lower())
    
    # tokenise
    words = text.split()
    
    # Checks it's a word and removes stop words
    words = [word for word in words if word not in stopword_list]
    
    # Create stemmer object
    stemmer = PorterStemmer()
    
    # Add lemmas
    lemmas = []
    for word in words:
        lemmas.append(stemmer.stem(word))
    
    return lemmas

---
### Initial attempt

What follows here was my initial attempt to create a tokeniser and classifier of the whiskies. For some reason, I got it into my head that I needed to use the custom vocabulary list and apply a vectoriser for nose, palate and finish, and then start reshaping the sparse matrices that came out of it by deleting empty columns. I later realised that this indicates a misunderstanding of what a sparse matrix is and how it is handled, and thus this was all a waste of time that I couldn't get to work inside a pipeline anyway, but I've included it here as it's somewhat interesting.

In [5]:
#vectoriser_with_wordlist = CountVectorizer(tokenizer=tokenise_and_stem_text, vocabulary=wordlist)

#X_nose = vectoriser_with_wordlist.fit_transform(df['nose'])
#X_palate = vectoriser_with_wordlist.fit_transform(df['palate'])
#X_finish = vectoriser_with_wordlist.fit_transform(df['finish'])

In [6]:
#for i in range(50):
#    print("There are {} columns in the Nose matrix with {} '1's in it.".format(
#    (X_nose[X_nose.sum(axis=0)==i]).shape[1], i))
#    print("There are {} columns in the Palate matrix with {} '1's in it.".format(
#    (X_palate[X_palate.sum(axis=0)==i]).shape[1], i))
#   print("There are {} columns in the Finish matrix with {} '1's in it.".format(
#   (X_finish[X_finish.sum(axis=0)==i]).shape[1], i))

Here I realised that you can dramatically cut down the size of the matrix by dropping any columns that have fewer than 4 hits in them. But even using a lower bound of 2 would substantially reduce their size. 

Again, this shows something of a misconception. Unique hits like this tend to be informationally _more_ valuable than the opposite. After all, the point is to discriminate, and something that uniquely picks out one whisky is fairly useful.

In [7]:
def create_smaller_matrices(matrix, lowerbound=0, wordlist=wordlist):
    '''
    INPUTS:
    matrix - matrix you wish to make smaller 
    lowerbound - inclusive. The smallest total number of hits in the column of the matrix that
                you'll allow. e.g. if there's only 1 review that mentions 'ablaze',
                the sum for axis=0 will be 1. Worth keeping? Probably not...
    wordlist - list of words that are indexed in the same way as matrix
    OUTPUTS:
    smaller_matrix - smaller matrix to use 
    smaller_list - smaller list of words.
    '''
    # Get indices of the columns we want to keep (i.e. where col sums >= lowerbound)
    row_index, col_index = np.where(matrix.sum(axis=0)>=lowerbound)
        
    smaller_list = []
    for i in range(len(col_index)):
        column_to_add = matrix.toarray()[:,col_index[i]].copy().reshape(-1, 1)
        if i == 0:
            # Create new_matrix with first column
            smaller_matrix = column_to_add
        else:
            # Concatenate new_matrix with i'th column. 
            smaller_matrix = np.hstack((smaller_matrix, column_to_add))
           
        # Add to newlist in order they appear in wordlist
        smaller_list.append(wordlist[col_index[i]])
        
    return smaller_matrix, smaller_list

In [8]:
# create smaller matrices for each in turn with a lowerbound of 3. 
#lb = 3
#nose_matrix, nose_list = create_smaller_matrices(X_nose, lowerbound=lb, wordlist=wordlist)
#palate_matrix, palate_list = create_smaller_matrices(X_palate, lowerbound=lb, wordlist=wordlist)
#finish_matrix, finish_list = create_smaller_matrices(X_finish, lowerbound=lb, wordlist=wordlist)

# Overlapping areas in terms of tasting notes between the three domains:
#overlap_list = sorted((set(nose_list) & set(palate_list)) & set(finish_list))
#nose_matrix.shape[1], palate_matrix.shape[1], finish_matrix.shape[1], len(overlap_list)

In [9]:
#nose_list = ['nose_'+item for item in nose_list]
#nose_list

My comment at the time was:

>Now I'm finally able to produce a matrix of tasting notes, separated by nose, palate and finish sections for each row of the df_collated dataframe. The overlap list might be helpful down the line if I want to perform analyses along the lines of 'Which whiskies are the most different from nose to palate?' or similar. The next tasks I need to perform are to hstack these matrices together to create a matrix of shape 27513 x 3052 (or whatever it is, depending on the hyperparameters used), and keep track of what each column means. Then we need to start training a Classifier using the matrix. 

>At this point, I need to write this as a proper function and put it inside a pipeline, because I'll need to split the data into a training and testing set in order to evaluate different potential classifiers. Technically, I already have a bit of data leakage here because I constructed my vocublary list using the full dataset. However, it's not a serious leakage as I was mainly just including as many tasting-y sounding words as possible in that wordlist. Since very rarely used words aren't going to be that important anyway, it's extremely unlikely that a vocab list generated from 20,000 reviews (say) as opposed to 25,000 would be that different.

The function below was intended simply to concatenate the reduced matrices for nose, palate and finish together, and keep track of what each column meant by creating a 'tasting note list'. 

In [10]:
def create_review_by_tasting_note_matrix(lowerbound=0):
    
    # create smaller matrices for each in turn with a lowerbound set to lowerbound. 
    lb = lowerbound
    nose_matrix, nose_list = create_smaller_matrices(X_nose, lowerbound=lb, wordlist=wordlist)
    palate_matrix, palate_list = create_smaller_matrices(X_palate, lowerbound=lb, wordlist=wordlist)
    finish_matrix, finish_list = create_smaller_matrices(X_finish, lowerbound=lb, wordlist=wordlist)
    
    # Overlapping areas in terms of tasting notes between the three domains:
    overlap_list = sorted((set(nose_list) & set(palate_list)) & set(finish_list))
    
    # Hstack the three matrices
    review_by_tasting_note_matrix = np.hstack((nose_matrix, palate_matrix, finish_matrix))
    
    # Create tasting_note_list
    nose_list = ['nose_'+item for item in nose_list]
    palate_list = ['palate_'+item for item in palate_list]
    finish_list = ['finish_'+item for item in palate_list]
    tasting_note_list = nose_list+palate_list+finish_list
    
    return review_by_tasting_note_matrix, tasting_note_list, overlap_list

# review_by_tasting_note_matrix, tasting_note_list, overlap_list = create_review_by_tasting_note_matrix(1)

At this point I realised this wasn't the right approach:

1. You shouldn't be eliminating columns with a few hits in them anyway.
2. Even deleting empty columns is basically a waste of time because of the way sparse matrices are handled by sklearn and scipy.
3. I needed to write a pipeline so that I could start trying out classifiers, using cross-validation and fine-tuning, etc., and my functions weren't going to play nice inside a pipeline.

---
### New Approach

I want to set up a pipeline now without any of this matrix reduction stuff, but simply employing three count-vectorisers in parallel for the nose, palate and finish respectively. I don't think there's going to be any way here to keep track of what each column means, but that's OK. Once the model is created and tuned and we know exactly what we want, we can always go back and build it manually and interpret it.

So we need three tiny, bespoke functions that simply grabs the correct column of data. Then we run three countvectorizers in parallel, then we bring them together using featureunion, run them through a TfIDF transformer, and then a classifier.

Below is an earlier pipeline I used to try out different classifiers. It was possible to get a small improvement by not using my vocabulary list, but that is effectively cheating since the review will often have the name of the whisky itself in the review, but I was interested to see how much difference it made. 

I tried out a whole bunch of classifiers with default values to get a sense of which ones would be best to have a go at fine-tuning. SVC, logistic regression and random forest came out well, but SGD and MLP also did fairly well. But because I didn't want a grid search taking forever, I ended up just looking for the best version of SVC, logistic regression and random forest. 

In [11]:
# Helper functions for getting the right column of data to use in the pipeline:
def getnose(array):
    return array[:,0]
    
def getpalate(array):
    return array[:,1]

def getfinish(array):
    return array[:,2]

# Earlier pipeline I used to try out different classifiers.
pipeline = Pipeline([
        
        ('union', FeatureUnion([
            
            ('pipe_nose', Pipeline([
                ('get_nose', FunctionTransformer(getnose)),
                #('countvec_nose', CountVectorizer(
                #    tokenizer=tokenise_and_stem_text) )
                ('countvec_nose', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
            ('pipe_palate', Pipeline([
                ('get_palate', FunctionTransformer(getpalate)),
                #('countvec_palate', CountVectorizer(
                #    tokenizer=tokenise_and_stem_text) )
                ('countvec_palate', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
            ('pipe_finish', Pipeline([
                ('get_finish', FunctionTransformer(getfinish)),
                #('countvec_finish', CountVectorizer(
                #    tokenizer=tokenise_and_stem_text) )
                ('countvec_finish', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
        ]) ),
         
        ('tfidf_transform', TfidfTransformer()),
        
        # Classifiers to try
        ('clf_RF', RandomForestClassifier(n_jobs=-1)) #19%
        #('clf_kneigh', KNeighborsClassifier()), #6%
        #('clf_svc', SVC()), #19%
        #('clf_gpc', GaussianProcessClassifier()), #didn't work
        #('clf_tree', DecisionTreeClassifier()), #14%
        #('clf_MLP', MLPClassifier()), #16%
        #('clf_Ada', AdaBoostClassifier()), #4%
        #('clf_NBayes', GaussianNB()), #didn't work
        #('clf_QuadDA', QuadraticDiscriminantAnalysis()) #didn't work
        #('clf_logreg', LogisticRegression()) #17%
        #('clf_SGDclf', SGDClassifier()) #15%
        ])

#X_train, X_test, y_train, y_test = train_test_split(df[['nose', 'palate', 'finish']], df['brand'])
#X_train, X_test = X_train.to_numpy(), X_test.to_numpy()

#pipeline.fit(X_train, y_train)

#y_pred = pipeline.predict(X_test)

#print(classification_report(y_test, y_pred, zero_division=0))

In [13]:
# Helper functions for getting the right column of data to use in the pipeline:
def getnose(array):
    return array[:,0]
    
def getpalate(array):
    return array[:,1]

def getfinish(array):
    return array[:,2]

# Pipeline to use
pipeline = Pipeline([
        
        ('union', FeatureUnion([
            
            ('pipe_nose', Pipeline([
                ('get_nose', FunctionTransformer(getnose)),
                ('countvec_nose', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
            ('pipe_palate', Pipeline([
                ('get_palate', FunctionTransformer(getpalate)),
                ('countvec_palate', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
            ('pipe_finish', Pipeline([
                ('get_finish', FunctionTransformer(getfinish)),
                ('countvec_finish', CountVectorizer(
                    tokenizer=tokenise_and_stem_text, vocabulary=wordlist) )
            ]) ),
        ]) ),
         
        ('tfidf_transform', TfidfTransformer()),
        
        ('clf', RandomForestClassifier(n_jobs=-1))
        ])

param_grid = [
    {'clf' : [LogisticRegression()],
     'clf__penalty' : ['l2'],
    'clf__C' : [1, 2, 4, 8],
    'clf__solver' : ['liblinear', 'sag']},
    {'clf' : [RandomForestClassifier(n_jobs=-1)],
    'clf__bootstrap': [False],
    'clf__max_depth': [200, 500],
    'clf__max_features': ['sqrt'],
    'clf__n_estimators': [1000, 2000]},
    {'clf' : [SVC()],
     'clf__degree' : [1, 2],
     'clf__C' : [2, 3, 4]}
]

# First of all, do the whole set of whiskies
X_train, X_test, y_train, y_test = train_test_split(
    df[['nose', 'palate', 'finish']], df['brand'])
X_train, X_test = X_train.to_numpy(), X_test.to_numpy()

grid_search_all_whiskies = GridSearchCV(estimator = pipeline, cv=3, 
                    param_grid = param_grid, scoring='accuracy', verbose=3)

grid_search_all_whiskies.fit(X_train, y_train)

y_pred = grid_search_all_whiskies.predict(X_test)

print(classification_report(y_test, y_pred, zero_division=0))

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.153 total time=  42.5s
[CV 2/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.154 total time=  42.0s
[CV 3/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.154 total time=  43.0s
[CV 1/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.155 total time=  40.5s
[CV 2/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.155 total time=  41.3s
[CV 3/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.155 total time=  46.0s
[CV 1/3] END clf=LogisticRegression(), clf__C=2, clf__penalty=l2, clf__solver=liblinear;, score=0.161 total time=  48.0s
[CV 2/3] END clf=LogisticRegression(), clf__C=2, clf__penalty=l2, clf__solver=liblinear;, score=0.163 total ti

C:\Users\jamie\anaconda3\envs\whisky_env\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 1/3] END clf=LogisticRegression(), clf__C=16, clf__penalty=l2, clf__solver=sag;, score=0.162 total time=  53.8s


C:\Users\jamie\anaconda3\envs\whisky_env\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 2/3] END clf=LogisticRegression(), clf__C=16, clf__penalty=l2, clf__solver=sag;, score=0.166 total time=  53.6s


C:\Users\jamie\anaconda3\envs\whisky_env\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV 3/3] END clf=LogisticRegression(), clf__C=16, clf__penalty=l2, clf__solver=sag;, score=0.168 total time=  53.7s
[CV 1/3] END clf=RandomForestClassifier(n_jobs=-1), clf__bootstrap=False, clf__max_depth=200, clf__max_features=sqrt, clf__n_estimators=1000;, score=0.182 total time= 3.4min
[CV 2/3] END clf=RandomForestClassifier(n_jobs=-1), clf__bootstrap=False, clf__max_depth=200, clf__max_features=sqrt, clf__n_estimators=1000;, score=0.186 total time= 3.4min
[CV 3/3] END clf=RandomForestClassifier(n_jobs=-1), clf__bootstrap=False, clf__max_depth=200, clf__max_features=sqrt, clf__n_estimators=1000;, score=0.185 total time= 3.4min
[CV 1/3] END clf=RandomForestClassifier(n_jobs=-1), clf__bootstrap=False, clf__max_depth=200, clf__max_features=sqrt, clf__n_estimators=2000;, score=0.182 total time= 8.9min
[CV 2/3] END clf=RandomForestClassifier(n_jobs=-1), clf__bootstrap=False, clf__max_depth=200, clf__max_features=sqrt, clf__n_estimators=2000;, score=0.183 total time= 9.3min
[CV 3/3] END c

In [14]:
# Save to pickle file
with open('./models/whisky_classifier.pkl', "wb") as f:
    joblib.dump(grid_search_all_whiskies, f, compress='zlib')

In [33]:
y_pred = grid_search_all_whiskies.predict(X_train)
print(classification_report(y_train, y_pred, zero_division=0))

                                         precision    recall  f1-score   support

                                   1792       1.00      0.97      0.99        72
                              Aberfeldy       1.00      0.93      0.97        46
                               Aberlour       0.97      0.96      0.97       317
                              Ailsa Bay       1.00      1.00      1.00         7
                                 Akashi       1.00      1.00      1.00        15
                                Alberta       1.00      1.00      1.00        30
                         Allt-A-Bhainne       1.00      1.00      1.00         9
                                  Amrut       0.99      0.98      0.99       169
                                 AnCnoc       0.98      0.89      0.93        93
                            Angels Envy       1.00      0.91      0.95        23
                        Angels Envy Rye       1.00      1.00      1.00        16
                           

In [3]:
# Testing out prediction code
test = np.array([['','',''],['fragrant, vanilla, melon, grass', 'grass, sherry, oaky', 'calm, balanced']])

grid_search_all_whiskies.predict(test)[1]

NameError: name 'np' is not defined

In [53]:
# Then do just the scotch single malts
X2_train, X2_test, y2_train, y2_test = train_test_split(
    df_sms[['nose', 'palate', 'finish']], df_sms['brand'])
X2_train, X2_test = X2_train.to_numpy(), X2_test.to_numpy()

grid_search_single_malts = GridSearchCV(estimator = pipeline, cv=3, 
                    param_grid = param_grid, scoring='accuracy', verbose=3)

grid_search_single_malts.fit(X2_train, y2_train)

y2_pred = grid_search_single_malts.predict(X2_test)

print(classification_report(y2_test, y2_pred, zero_division=0))

# Save to pickle file
with open('./models/malt_classifier.pkl', "wb") as f:
    joblib.dump(grid_search_single_malts, f, compress='zlib')

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV 1/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.184 total time=  25.5s
[CV 2/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.177 total time=  25.7s
[CV 3/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=liblinear;, score=0.170 total time=  25.5s
[CV 1/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.185 total time=  26.0s
[CV 2/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.178 total time=  26.1s
[CV 3/3] END clf=LogisticRegression(), clf__C=1, clf__penalty=l2, clf__solver=sag;, score=0.170 total time=  26.6s
[CV 1/3] END clf=LogisticRegression(), clf__C=2, clf__penalty=l2, clf__solver=liblinear;, score=0.190 total time=  28.5s
[CV 2/3] END clf=LogisticRegression(), clf__C=2, clf__penalty=l2, clf__solver=liblinear;, score=0.183 total ti

In the end, I decided not to proceed with the 'Just single malts' version. It did slightly better, with 23% accuracy as opposed to 21%, but not enough to justify the exclusion of so many interesting whiskies that make the app a truly international affair.

As for the score itself, I doubt anyone is going to be blown away by an accuracy of 21%, but informal feedback from people who know their whisky is fairly positive. Even when the guesses are wrong, they make sense. The model is producing a multi-dimensional matrix where the Euclidean distance does seem to correspond more or less to one's intuitive sense of which whiskies are similar to each other, which is a good sign. 